# Imports and General Settings

In [1]:
import tensorflow as tf
import tensorflow.contrib as tf_contrib

import os
import pathlib

import numpy as np

import io

import random

import pylab
import tfplot

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import tfx
import tensorflow_transform as tft

from cleverhans.loss import SNNLCrossEntropy

# tensorflow specific settings
tf.enable_eager_execution()
tf.logging.set_verbosity(tf.logging.DEBUG)

# which GPU to be used (0 is RTX, 1 or 2 are either of the Titan Xps)
gpu = "/GPU:0"

AUTOTUNE = tf.contrib.data.AUTOTUNE

session_config = tf.ConfigProto()
session_config.gpu_options.allow_growth = True
session_config.allow_soft_placement = True

# session_config.log_device_placement = True
tf.keras.backend.set_session(tf.Session(config=session_config))

# make sure tensorflow-gpu is being used
print(tf.test.gpu_device_name())

/usr/local/lib/python3.6/dist-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


/device:GPU:0


# Model Utility Functions

In [2]:

# Xavier : tf_contrib.layers.xavier_initializer()
# He : tf_contrib.layers.variance_scaling_initializer()
# Normal : tf.random_normal_initializer(mean=0.0, stddev=0.02)
# l2_decay : tf_contrib.layers.l2_regularizer(0.0001)

weight_init = tf_contrib.layers.variance_scaling_initializer()
weight_regularizer = tf_contrib.layers.l2_regularizer(0.0001)


##################################################################################
# Layer
##################################################################################

def conv(x, channels, kernel=4, stride=2, padding='SAME', use_bias=True, scope='conv_0'):
    with tf.variable_scope(scope):
        x = tf.layers.conv2d(inputs=x, filters=channels,
                             kernel_size=kernel, kernel_initializer=weight_init,
                             kernel_regularizer=weight_regularizer,
                             strides=stride, use_bias=use_bias, padding=padding)

        return x

def fully_connected(x, units, use_bias=True, scope='fully_0'):
    with tf.variable_scope(scope):
        x = flatten(x)
        x = tf.layers.dense(x, units=units, kernel_initializer=weight_init, kernel_regularizer=weight_regularizer, use_bias=use_bias)

        return x

def resblock(x_init, channels, kernel=3, is_training=True, use_bias=True, downsample=False, scope='resblock') :
    with tf.variable_scope(scope) :

        x = batch_norm(x_init, is_training, scope='batch_norm_0')
        x = relu(x)

        if downsample :
            x = conv(x, channels, kernel=kernel, stride=2, use_bias=use_bias, scope='conv_0')
            x_init = conv(x_init, channels, kernel=1, stride=2, use_bias=use_bias, scope='conv_init')

        else :
            x = conv(x, channels, kernel=kernel, stride=1, use_bias=use_bias, scope='conv_0')

        x = batch_norm(x, is_training, scope='batch_norm_1')
        x = relu(x)
        x = conv(x, channels, kernel=kernel, stride=1, use_bias=use_bias, scope='conv_1')

        return x + x_init

def bottle_resblock(x_init, channels, is_training=True, use_bias=True, downsample=False, scope='bottle_resblock') :
    with tf.variable_scope(scope) :
        x = batch_norm(x_init, is_training, scope='batch_norm_1x1_front')
        shortcut = relu(x)

        x = conv(shortcut, channels, kernel=1, stride=1, use_bias=use_bias, scope='conv_1x1_front')
        x = batch_norm(x, is_training, scope='batch_norm_3x3')
        x = relu(x)

        if downsample :
            x = conv(x, channels, kernel=3, stride=2, use_bias=use_bias, scope='conv_0')
            shortcut = conv(shortcut, channels*4, kernel=1, stride=2, use_bias=use_bias, scope='conv_init')

        else :
            x = conv(x, channels, kernel=3, stride=1, use_bias=use_bias, scope='conv_0')
            shortcut = conv(shortcut, channels * 4, kernel=1, stride=1, use_bias=use_bias, scope='conv_init')

        x = batch_norm(x, is_training, scope='batch_norm_1x1_back')
        x = relu(x)
        x = conv(x, channels*4, kernel=1, stride=1, use_bias=use_bias, scope='conv_1x1_back')

        return x + shortcut


##################################################################################
# Sampling
##################################################################################

def flatten(x) :
    return tf.layers.flatten(x)

def global_avg_pooling(x):
    gap = tf.reduce_mean(x, axis=[1, 2], keepdims=True)
    return gap

def avg_pooling(x) :
    return tf.layers.average_pooling2d(x, pool_size=2, strides=2, padding='SAME')

def max_pooling(x) :
    return tf.layers.max_pooling2d(x, pool_size=2, strides=2, padding='SAME')

##################################################################################
# Activation function
##################################################################################

def relu(x):
    return tf.nn.relu(x)


##################################################################################
# Normalization function
##################################################################################

def batch_norm(x, is_training=True, scope='batch_norm'):
    return tf_contrib.layers.batch_norm(x,
                                        decay=0.9, epsilon=1e-05,
                                        center=True, scale=True, updates_collections=None,
                                        is_training=is_training, scope=scope)

##################################################################################
# Loss function
##################################################################################

def classification_loss(logit, label) :
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=label, logits=logit))
    prediction = tf.equal(tf.argmax(logit, -1), tf.argmax(label, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

    return loss, accuracy


# Loading Data

## Data Augmentation Functions

In [3]:

def rotate(image):
    
    return tf.image.rot90(image, tf.random_uniform(shape=[], minval=0, maxval=4, 
                                               dtype=tf.int32))

def flip(image):
    
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)

    return image

def crop(image, range_start=0.8, range_end=1.0):
    
    image = tf.image.central_crop(image, central_fraction=random.uniform(range_start, range_end))
    image = tf.image.resize_images(image,tf.constant([512, 512]))
    return image

def translate(image,  x_max=75, y_max=75):
    
    return tf.contrib.image.translate(image, translations=[random.uniform(-1 * x_max, x_max), 
                                                           random.uniform(-1 * y_max, y_max)])

def augment_image(image):
                       
    image = rotate(image)
    image = flip(image)
    
    image = translate(image)
    image = crop(image)

    return image

## Importing Data (PNG)

Given a directory of data of the form: ~/data/labels/images.png

In [4]:
# load an array of image paths
def load_image_paths(path):
    
    data_root = pathlib.Path(path)
    
    # create a list of every file and its label index
    all_image_paths = list(data_root.glob('*/*'))
    all_image_paths = [str(path) for path in all_image_paths]
    
    return all_image_paths

# randomly shuffle train and test values and split based on parameters
def split(image_paths, split=[0.6,0.2,0.2], seed=777):
    random.Random(seed).shuffle(image_paths)
    
    boundary1 = int(len(image_paths) * split[0])
    boundary2 = int(len(image_paths) * (split[0]+split[1]))
    
    train = image_paths[:boundary1]
    evaluate = image_paths[boundary1: boundary2]
    test = image_paths[boundary2:]
    
    return train, evaluate, test

# preprocessing functions
def preprocess_image(image):
    image = tf.image.decode_png(image, channels=1)
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, shape=[512,512])
    image = tf.div(
       tf.subtract(
          image, 
          tf.reduce_min(image)
       ), 
       tf.subtract(
          tf.reduce_max(image), 
          tf.reduce_min(image)
       )
    )
    
    return image

def load_and_preprocess_image(path, training=True):
    image = tf.read_file(path)
    image = preprocess_image(image)
    
    if (training):
        image = augment_image(image[:,:,None])
    
    return image

def load(path, image_paths, training=True, batch_size=64, shuffle=True, drop_remainder=False):
    with tf.device("/CPU:0"):
        # data root
        data_root = pathlib.Path(path)

        # return label names
        label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())

        # assign index to label
        label_to_index = dict((name, index) for index,name in enumerate(label_names))

        # array of all labels corresponding to image_paths
        all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                            for path in image_paths]

        # make path dataset
        path_ds = tf.data.Dataset.from_tensor_slices(image_paths)

        # get image tensors by mapping function over the path dataset
        image_ds = path_ds.map(lambda path : load_and_preprocess_image(path, training=training))

        # create label dataset
        label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels, tf.int64))

        # zip together image and label dataset into dataset of tuples for
        # estimator input
        image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))

        # Setting a shuffle buffer size as large as the dataset ensures that the data is
        # completely shuffled
        
        image_label_ds = image_label_ds.batch(batch_size, drop_remainder=drop_remainder)
        
        if (training):
            print("shuffling and repeating b/c training flag set")

            image_label_ds = image_label_ds.repeat()
        else:
            image_label_ds = image_label_ds.repeat(count=1)
        
        
        if (shuffle):
            image_label_ds = image_label_ds.shuffle(buffer_size = len(image_paths))
            
        # `prefetch` lets the dataset fetch batches, in the background while the model is training.
        image_label_ds = image_label_ds.prefetch(6)
       
        
        return image_label_ds

# ResNet-10

## Logging Utility Functions

In [5]:
# adapted from https://github.com/grishasergei/conviz/
def prime_powers(n):
    """
    Compute the factors of a positive integer
    Algorithm from https://rosettacode.org/wiki/Factors_of_an_integer#Python
    :param n: int
    :return: set
    """
    factors = set()
    for x in range(1, int(np.sqrt(n)) + 1):
        if n % x == 0:
            factors.add(int(x))
            factors.add(int(n // x))
    return sorted(factors)

def get_grid_dim(x):
    """
    Transforms x into product of two integers
    :param x: int
    :return: two ints
    """
    factors = prime_powers(x)
    if len(factors) % 2 == 0:
        i = int(len(factors) / 2)
        return factors[i], factors[i - 1]

    i = len(factors) // 2
    return factors[i], factors[i]

def pca(X, num_observations=64, n_dimensions = 50):
    singular_values, u, _ = tf.svd(X)
    sigma = tf.diag(singular_values)
    print(sigma)
    
    sigma = tf.slice(sigma, [0, 0], [num_observations, n_dimensions])
    
    pca = tf.matmul(u, sigma)
    pca = tf.transpose(pca)
    return pca

## Logging Convolution Filters

In [6]:
@tfplot.wrap
def plot_conv_weights(weights, channels_all=True):
    """
    Plots convolutional filters
    :param weights: numpy array of rank 4
    :param channels_all: boolean, optional

    """
    w_min = np.min(weights)
    w_max = np.max(weights)

    channels = [0]
    # make a list of channels if all are plotted
    if channels_all:
        channels = range(weights.shape[2])

    # get number of convolutional filters
    num_filters = weights.shape[3]

    # get number of grid rows and columns
    grid_r, grid_c = get_grid_dim(num_filters)
    
    # create figure and axes
    fig, axes = plt.subplots(min([grid_r, grid_c]), max([grid_r, grid_c]))

    # iterate channels
    for channel in channels:
        # iterate filters inside every channel
        for l, ax in enumerate(axes.flat):
            # get a single filter
            img = weights[:, :, channel, l]
            # put it on the grid
            ax.imshow(img, vmin=w_min, vmax=w_max, interpolation='nearest', cmap='seismic')
            
            # remove any labels from the axes
            ax.set_xticks([])
            ax.set_yticks([])
            
    plt.close(fig)
    
    return fig



## Logging Activation Maps

In [7]:
@tfplot.wrap
def plot_conv_output(conv_img, name):
    """
    Makes plots of results of performing convolution
    :param conv_img: numpy array of rank 4
    :param name: string, name of convolutional layer
    :return: nothing, plots are saved on the disk
    """
    w_min = np.min(conv_img)
    w_max = np.max(conv_img)

    # get number of convolutional filters
    num_filters = conv_img.shape[3]

    # get number of grid rows and columns
    grid_r, grid_c = get_grid_dim(num_filters)
    

    if (min([grid_r, grid_c]) == 1 and max([grid_r, grid_c]) == 1):
        
        fig, ax = plt.subplots(figsize=(24,24))   
        fig.suptitle(name, fontsize=25, color='k')
        fig.tight_layout(rect=[0, 0.03, 1, 0.95])
        
        img = conv_img[0, :, :,  0]
        ax.imshow(img, vmin=w_min, vmax=w_max, interpolation='bicubic', cmap='Greys_r')
        
        ax.set_xticks([])
        ax.set_yticks([])
        
        plt.subplots_adjust(top=0.85)
        plt.close(fig)
        return fig
        
        
    # create figure and axes
    fig, axes = plt.subplots(min([grid_r, grid_c]),
                             max([grid_r, grid_c]),
                            figsize=(24,15))
    fig.suptitle(name, fontsize=25, color='k')
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    
    # iterate filters
    for l, ax in enumerate(axes.flat):
        # get a single image
        img = conv_img[0, :, :,  l]
        # put it on the grid
        ax.imshow(img, vmin=w_min, vmax=w_max, interpolation='bicubic', cmap='jet')
        # remove any labels from the axes
        ax.set_xticks([])
        ax.set_yticks([])
       
    plt.close(fig)
    return fig
    
    

## Architecture

In [8]:
num_classes = 3

# ResNet-10 architecture
def resnet10_network(x, conv_kernel=7, resblock_kernel=3, num_channels=4, n_classes=num_classes, reuse = False, is_training = True):
    with tf.variable_scope('Resnet-10', reuse = reuse), tf.device(gpu):
        
        x = tf.reshape(x, shape=[-1, 512, 512, 1])
        # channels
        
        filtered_images = []
        
        filtered_images.append(tf.identity(x))
        
        x = conv(x, num_channels, kernel=conv_kernel, stride=1, padding='SAME')
        
        filtered_images.append(tf.identity(x))
        
        x = resblock(x, channels=num_channels, kernel=resblock_kernel, is_training=is_training, 
                      downsample=False, scope='resblock1_id1')      
   
        filtered_images.append(tf.identity(x))

        x = resblock(x, channels=num_channels * 2, kernel=resblock_kernel, is_training=is_training, 
                     downsample=True, scope='resblock2_conv1')
        
        filtered_images.append(tf.identity(x))

        x = resblock(x, channels=num_channels * 2, kernel=resblock_kernel, is_training=is_training, 
                      downsample=False, scope='resblock3_id2')
        
        filtered_images.append(tf.identity(x))
        
        x = resblock(x, channels=num_channels * 4, kernel=resblock_kernel, is_training=is_training, 
                     downsample=True, scope='resblock4_conv2')

        filtered_images.append(tf.identity(x))
        
        x = resblock(x, channels=num_channels * 4, kernel=resblock_kernel, is_training=is_training, 
                      downsample=False, scope='resblock5_id3')
        
        filtered_images.append(tf.identity(x))
        
        
        x = batch_norm(x, is_training, scope='batch_norm')
        
        x = relu(x)
        
        x_pre_flatten = tf.identity(x)
        
        x = fully_connected(x, num_classes)
        
        return x, x_pre_flatten, filtered_images

    
def resnet10_model_fn(features, labels, mode, conv_kernel=7, resblock_kernel=3, num_channels=4, snnl_weight=0.05,
                      log=True):
    with tf.device(gpu):

        print("in gpu part")
        logits_train, activ_maps_train, filtered_images = resnet10_network(features,
                                                          conv_kernel=conv_kernel,
                                                          resblock_kernel=resblock_kernel,
                                                          num_channels=num_channels,
                                                          n_classes=num_classes,
                                                          reuse=False,
                                                          is_training=True)

        logits_test, activ_maps_test, _ = resnet10_network(features,
                                                        conv_kernel=conv_kernel,
                                                        resblock_kernel=resblock_kernel,
                                                        num_channels=num_channels,
                                                        n_classes=num_classes,
                                                        reuse=True,
                                                        is_training=False)

        pred_classes = tf.arg_max(logits_test, dimension=1)

        # PREDICT MODE
        if mode == tf.estimator.ModeKeys.PREDICT:

            return tf.estimator.EstimatorSpec(mode=mode, predictions=pred_classes)


        loss_op = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits_train, labels=tf.cast(labels, dtype=tf.int32)))

        loss_op = (1.0 - snnl_weight) * loss_op + snnl_weight * SNNLCrossEntropy.optimized_temp_SNNL(
            flatten(activ_maps_train), labels, 20.0, True)
        
        acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)

        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
        
        tf.summary.scalar("loss", loss_op)
        tf.summary.scalar("accuracy", acc_op[1])
        
        if mode == tf.estimator.ModeKeys.TRAIN:

            # the different ops for training, evaluating, ...
            estim_specs = tf.estimator.EstimatorSpec(
                mode=mode,
                predictions=pred_classes,
                loss=loss_op,
                train_op=train_op,
                eval_metric_ops={'accuracy': acc_op}
            )

            return estim_specs
    
#     for var in tf.trainable_variables():
#         print(var.name)
    
        conv0_weights = [v for v in tf.global_variables() if v.name == "Resnet-10/conv_0/conv2d/kernel:0"][0]
        
        plot_op_expanded = tf.expand_dims(plot_conv_weights(conv0_weights), 0)
        tf.summary.image("conv_0_weight_plots", plot_op_expanded)
        
        
        with tf.name_scope("activation_maps"):
            
            level_dict = {0: "0_original_image",         
                          1 : "1_after_initial_conv",
                          2 : "2_after_resblock1_id1",
                          3 : "3_after_resblock2_conv1",
                          4 : "4_after_resblock3_id2",
                          5 : "5_after_resblock4_conv2",
                          6 : "6_after_resblock5_id3"}
            
            counter = 0
            
            plot_ops = []

            for filtered_im in filtered_images:
               
                activation_map = tf.slice(filtered_im,[0, 0, 0, 0],[1, -1, -1, -1])
                activation_op = tf.expand_dims(plot_conv_output(activation_map, level_dict[counter]), 0)

                plot_ops.append(activation_op)

                counter = counter + 1
                
            final_log_op = tf.concat(plot_ops, 1)
        
            tf.summary.image("image_display", final_log_op)
        
        
#         level_dict = {0: "0_original_image",
            
#                       1 : "1_after_initial_conv",
#                       2 : "2_after_resblock1_id1",
#                       3 : "3_after_resblock2_conv1",
#                       4 : "4_after_resblock3_id2",
#                       5 : "5_after_resblock4_conv2",
#                       6 : "6_after_resblock5_id3"}
        
#         with tf.name_scope("activation_maps"):
            
#             counter = 0
            

#             for filtered_im in filtered_images:
            
#                 with tf.name_scope(level_dict[counter]): 
#                     activation_map = tf.slice(filtered_im,[0, 0, 0, 0],[1, -1, -1, -1])
#                     activation_op = tf.expand_dims(plot_conv_output(activation_map), 0)
                    
#                     print(activation_op)
                    
#                     tf.summary.image("filtered_images", activation_op)

#                 counter = counter + 1
            
        eval_summary_hook = tf.train.SummarySaverHook(
            save_steps=1,
            output_dir=model_path + "/eval_images",
            summary_op=tf.summary.merge_all())
        # Add it to the evaluation_hook list
        evaluation_hooks = []
        evaluation_hooks.append(eval_summary_hook)

        # the different ops for training, evaluating, ...
        estim_specs = tf.estimator.EstimatorSpec(
            mode=mode,
            loss=loss_op,
            train_op=train_op,
            eval_metric_ops={'accuracy': acc_op},
            evaluation_hooks=evaluation_hooks
        )

        return estim_specs

# Running Results

## Experimentation

In [ ]:

tf.logging.set_verbosity(tf.logging.INFO)

b_size = 32

learning_rate = 0.005

random_seed = 777

num_steps = 10000

resnet10_checkpoint_path_base = "/data/jma/model/resnet10_test/"

# kernel_sizes = [7, 9, 10, 11, 13]
# filter_sizes = [10]
# snnl_weights = [0.0]

kernel_sizes = [3, 5, 7]
filter_sizes = [4]
snnl_weights = [0.05]

data_root = "./data_3classes/"

image_paths = load_image_paths(data_root)
train_paths, eval_paths, test_paths = split(image_paths, seed=random_seed)

train_input_fn = lambda : load(data_root, train_paths, training=True, batch_size=32, shuffle=False)
eval_input_fn = lambda : load(data_root, eval_paths, training=False, batch_size=32, shuffle=False)
test_input_fn = lambda : load(data_root, test_paths, training=False, shuffle=False)

for filter_size in filter_sizes:
    for kernel_size in kernel_sizes:
        
        for snnl_weight in snnl_weights:
            tf.summary.FileWriterCache.clear()
 
            model_path = resnet10_checkpoint_path_base + "kernel" + str(kernel_size) + "_filter" + str(filter_size) + "_snnl" + str(snnl_weight)+"/"
            print(model_path)

            config = tf.estimator.RunConfig(
                log_step_count_steps= 20,
                save_summary_steps= 20,
                save_checkpoints_secs= 120,
                model_dir=model_path,
                tf_random_seed=random_seed,
                session_config=session_config)

            model_fn = lambda features, labels, mode: resnet10_model_fn(features, labels, mode, 
                                                                        conv_kernel=kernel_size, 
                                                                        num_channels=filter_size,
                                                                        snnl_weight=snnl_weight)

            model = tf.estimator.Estimator(model_fn=model_fn, config=config)

            train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=num_steps)
            eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn, throttle_secs=15)

            tf.estimator.train_and_evaluate(model, train_spec, eval_spec)


        
        

/data/jma/model/resnet10_test/kernel3_filter4_snnl0.05/
INFO:tensorflow:Using config: {'_model_dir': '/data/jma/model/resnet10_test/kernel3_filter4_snnl0.05/', '_tf_random_seed': 777, '_save_summary_steps': 20, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 120, '_session_config': gpu_options {
  allow_growth: true
}
allow_soft_placement: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 20, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f74a076d4a8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The e

INFO:tensorflow:global_step/sec: 2.52351
INFO:tensorflow:loss = 0.10672902, step = 820 (7.925 sec)
INFO:tensorflow:global_step/sec: 2.49267
INFO:tensorflow:loss = 0.18545239, step = 840 (8.026 sec)
INFO:tensorflow:global_step/sec: 2.4935
INFO:tensorflow:loss = 0.41543674, step = 860 (8.020 sec)
INFO:tensorflow:global_step/sec: 2.63758
INFO:tensorflow:loss = 0.14574456, step = 880 (7.584 sec)
INFO:tensorflow:Saving checkpoints for 889 into /data/jma/model/resnet10_test/kernel3_filter4_snnl0.05/model.ckpt.
INFO:tensorflow:Calling model_fn.
in gpu part
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-07-29-20:51:15
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /data/jma/model/resnet10_test/kernel3_filter4_snnl0.05/model.ckpt-889
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-07-29-20:52:00
INFO:tensorflow:Saving dict for global step 889: accura

INFO:tensorflow:global_step/sec: 0.38833
INFO:tensorflow:loss = 0.2099092, step = 1680 (51.502 sec)
INFO:tensorflow:global_step/sec: 2.5376
INFO:tensorflow:loss = 0.08586694, step = 1700 (7.881 sec)
INFO:tensorflow:global_step/sec: 2.54221
INFO:tensorflow:loss = 0.20494062, step = 1720 (7.868 sec)
INFO:tensorflow:global_step/sec: 2.5422
INFO:tensorflow:loss = 0.09881458, step = 1740 (7.867 sec)
INFO:tensorflow:global_step/sec: 2.65888
INFO:tensorflow:loss = 0.1764813, step = 1760 (7.522 sec)
INFO:tensorflow:global_step/sec: 2.55441
INFO:tensorflow:loss = 0.08669348, step = 1780 (7.829 sec)
INFO:tensorflow:global_step/sec: 2.53069
INFO:tensorflow:loss = 0.08670075, step = 1800 (7.903 sec)
INFO:tensorflow:global_step/sec: 2.54211
INFO:tensorflow:loss = 0.23707499, step = 1820 (7.868 sec)
INFO:tensorflow:global_step/sec: 2.64857
INFO:tensorflow:loss = 0.1377434, step = 1840 (7.551 sec)
INFO:tensorflow:global_step/sec: 2.50619
INFO:tensorflow:loss = 0.07192959, step = 1860 (7.983 sec)
INFO

INFO:tensorflow:Starting evaluation at 2019-07-29-21:09:17
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /data/jma/model/resnet10_test/kernel3_filter4_snnl0.05/model.ckpt-2638
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-07-29-21:09:58
INFO:tensorflow:Saving dict for global step 2638: accuracy = 0.5341615, global_step = 2638, loss = 1.2659384
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2638: /data/jma/model/resnet10_test/kernel3_filter4_snnl0.05/model.ckpt-2638
INFO:tensorflow:global_step/sec: 0.396459
INFO:tensorflow:loss = 0.08493774, step = 2640 (50.447 sec)
INFO:tensorflow:global_step/sec: 3.08894
INFO:tensorflow:loss = 0.11003022, step = 2660 (6.475 sec)
INFO:tensorflow:global_step/sec: 2.52146
INFO:tensorflow:loss = 0.1464166, step = 2680 (7.934 sec)
INFO:tensorflow:global_step/sec: 2.5259
INFO:tensorflow:loss = 0.13404767, step = 2700 (7.915 sec

INFO:tensorflow:global_step/sec: 2.65436
INFO:tensorflow:loss = 0.12674932, step = 3520 (7.534 sec)
INFO:tensorflow:global_step/sec: 2.51079
INFO:tensorflow:loss = 0.08560735, step = 3540 (7.965 sec)
INFO:tensorflow:global_step/sec: 2.5212
INFO:tensorflow:loss = 0.13900095, step = 3560 (7.935 sec)
INFO:tensorflow:global_step/sec: 2.52475
INFO:tensorflow:loss = 0.07555066, step = 3580 (7.921 sec)
INFO:tensorflow:global_step/sec: 2.59664
INFO:tensorflow:loss = 0.07918551, step = 3600 (7.702 sec)
INFO:tensorflow:Saving checkpoints for 3612 into /data/jma/model/resnet10_test/kernel3_filter4_snnl0.05/model.ckpt.
INFO:tensorflow:Calling model_fn.
in gpu part
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-07-29-21:19:18
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /data/jma/model/resnet10_test/kernel3_filter4_snnl0.05/model.ckpt-3612
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:ten

INFO:tensorflow:Finished evaluation at 2019-07-29-21:28:01
INFO:tensorflow:Saving dict for global step 4403: accuracy = 0.5031056, global_step = 4403, loss = 1.4232364
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 4403: /data/jma/model/resnet10_test/kernel3_filter4_snnl0.05/model.ckpt-4403
INFO:tensorflow:global_step/sec: 0.403632
INFO:tensorflow:loss = 0.09853714, step = 4420 (49.550 sec)
INFO:tensorflow:global_step/sec: 2.54454
INFO:tensorflow:loss = 0.07416017, step = 4440 (7.860 sec)
INFO:tensorflow:global_step/sec: 2.54047
INFO:tensorflow:loss = 0.2622909, step = 4460 (7.873 sec)
INFO:tensorflow:global_step/sec: 2.64741
INFO:tensorflow:loss = 0.09779388, step = 4480 (7.556 sec)
INFO:tensorflow:global_step/sec: 2.50562
INFO:tensorflow:loss = 0.11245553, step = 4500 (7.981 sec)
INFO:tensorflow:global_step/sec: 2.52547
INFO:tensorflow:loss = 0.10618608, step = 4520 (7.919 sec)
INFO:tensorflow:global_step/sec: 2.51563
INFO:tensorflow:loss = 0.26031974, step = 4540 (

# Metrics

## Metric Utility Functions

In [17]:
def path_to_label(paths, data_root):

    data_root = pathlib.Path(data_root)

    # return label names
    label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())

    # assign index to label
    label_to_index = dict((name, index) for index,name in enumerate(label_names))

    # array of all labels corresponding to image_paths
    all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                        for path in paths]
    
    return all_image_labels


## Confusion Matrices

In [ ]:
%matplotlib inline

tf.logging.set_verbosity(tf.logging.ERROR)

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

resnet10_checkpoint_path_base = "/data/jma/model/resnet10_test/"

kernel_sizes = [3,5,7]
filter_sizes = [8]
snnl_weights = [0.05]

data_path = "./data_3classes/"

random_seed = 777

image_paths = load_image_paths(data_path)
train_paths, eval_paths, test_paths = split(image_paths, seed=random_seed)

train_input_fn = lambda : load(data_path, train_paths, training=True, shuffle=True)
eval_input_fn = lambda : load(data_path, eval_paths, training=False, shuffle=False, batch_size=len(eval_paths))
test_input_fn = lambda : load(data_path, test_paths, training=False)

for filter_size in filter_sizes:
    for kernel_size in kernel_sizes:
        
        for snnl_weight in snnl_weights:
            tf.summary.FileWriterCache.clear()

            title = "kernel" + str(kernel_size) + "_filter" + str(filter_size) + "_snnl" + str(snnl_weight)
            
            model_path = resnet10_checkpoint_path_base + title +"/"
            print(model_path)
        
        
            model_fn = lambda features, labels, mode: resnet10_model_fn(features, labels, mode, 
                                                                        conv_kernel=kernel_size, 
                                                                        num_channels=filter_size)
            model = tf.estimator.Estimator(model_fn=model_fn, model_dir=model_path)

            values = model.predict(eval_input_fn)


            original_labels = path_to_label(eval_paths, data_path)
            predicted_labels = list(values)

            cm = tf.confusion_matrix(original_labels, predicted_labels)
            cm = tf.to_float(cm)

            cm = cm.numpy()
            
            row_sums = cm.sum(axis=1)
            cm = cm / row_sums[:, np.newaxis]
            
            df_cm = pd.DataFrame(cm, range(3), range(3))
            #plt.figure(figsize = (10,7))



            sns.set(font_scale=1)#for label size
            plt.suptitle("kernel" + str(kernel_size) + "_filter" + str(filter_size))
            ax = sns.heatmap(df_cm, annot=True,annot_kws={"size": 16})# font size
            ax.set(xlabel='true', ylabel='predicted')
            plt.show()
        

## Incorrectly Classified Images (PNG Format)

In [ ]:
%matplotlib inline

tf.logging.set_verbosity(tf.logging.ERROR)

import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

resnet10_checkpoint_path_base = "/home_local/jma/model/resnet10/"

kernel_sizes = [3,4,5,6,7,8,9,19]
filter_sizes = [4, 10, 16]

data_path = "./data_3classes/"

image_paths = load_image_paths(data_path)
train_paths, eval_paths, test_paths = split(image_paths, seed=random_seed)


train_input_fn = lambda : load(data_path, train_paths, training=True)
eval_input_fn = lambda : load(data_path, eval_paths, training=False)
test_input_fn = lambda : load(data_path, test_paths, training=False)

# change these======
working_paths = eval_paths
working_input_fn = eval_input_fn

kernel_size=8
filter_size=10
#=========

model_path = resnet10_checkpoint_path_base + "kernel" + str(kernel_size) + "_filter" + str(filter_size) + "/"
print(model_path)

model_fn = lambda features, labels, mode: resnet10_model_fn(features, labels, mode, 
                                                            conv_kernel=kernel_size, 
                                                            num_channels=filter_size)
model = tf.estimator.Estimator(model_fn=model_fn, model_dir=model_path)

values = model.predict(working_input_fn)

original_labels = path_to_label(working_paths, data_path)
predicted_labels = list(values)

# make a plot?

for index in range(len(working_paths)):
    if original_labels[index] != predicted_labels[index]:
        # add image to plot

        img = Image.open(working_paths[index])
        
        basewidth = 256

        wpercent = (basewidth/float(img.size[0]))
        hsize = int((float(img.size[1])*float(wpercent)))
        img = img.resize((basewidth,hsize), Image.ANTIALIAS)

        
        d = ImageDraw.Draw(img)
        
        # debug this shit
        d.text((10,10), "True: " + str(original_labels[index]) + 
               ", Pred: " + str(predicted_labels[index]), 
               fill="white", font=ImageFont.truetype("./fonts/arial-bold.ttf", 15))       
                
        display(img)
    